In [75]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, Input, regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MSE, MAE, BinaryCrossentropy

from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import pandas as pd
from aux_func import show_history, r2_keras
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.python.keras.backend import sigmoid
from tensorflow.data import Dataset

In [76]:
def custom_loss(alpha):
  def custom_loss_fun(y_true, y_pred):
    bce=BinaryCrossentropy(from_logits=False)
    return bce(y_true, y_pred[:,0])*alpha + MSE(y_true, y_pred[:,1])  
  return custom_loss_fun

In [77]:
# Esta función genera una serie temporal seudo aleatoria con cierta periodicidad
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [78]:
# Generamos train, validation y test
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train_sequence, y_train_sequence = series[:7000, :n_steps], series[:7000, -1]
X_valid_sequence, y_valid_sequence = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test_sequence, y_test_sequence = series[9000:, :n_steps], series[9000:, -1]
ytrain2=np.where(y_train_sequence-X_train_sequence[:,-1]>=0,1,0)
yvalid2=np.where(y_valid_sequence-X_valid_sequence[:,-1]>=0,1,0)
ytest2=np.where(y_test_sequence-X_test_sequence[:,-1]>=0,1,0)
y_train=np.concatenate((y_train_sequence,ytrain2), axis=-1)
y_valid=np.concatenate((y_valid_sequence,yvalid2), axis=-1)
y_test=np.concatenate((y_test_sequence,ytest2), axis=-1)

In [79]:
input_tensor = Input(shape=(n_steps), dtype='float32')
x_0 = input_tensor
x = tf.expand_dims(x_0, axis=-1)
# Dependiendo de cual sea el output deberemos cambiar el valor 'return_sequences'
for _ in range(5):
    x = LSTM(100, activation='tanh', recurrent_activation='sigmoid', return_sequences=True)(x)
x = LSTM(200, activation='tanh', recurrent_activation='sigmoid', return_sequences=False)(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation=tf.nn.leaky_relu)(x)
x = layers.Dense(100, activation=tf.nn.leaky_relu)(x)
out_reg = layers.Dense(1, activation=None)(x)
out_class= layers.Dense(1, activation='softmax')(x)
output = tf.concat([out_reg,out_class], axis=-1)
model = Model(input_tensor, output)
print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=custom_loss(1000),metrics=['mse', 'mean_absolute_error',r2_keras])

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 tf.expand_dims_15 (TFOpLambda)  (None, 50, 1)       0           ['input_16[0][0]']               
                                                                                                  
 lstm_90 (LSTM)                 (None, 50, 100)      40800       ['tf.expand_dims_15[0][0]']      
                                                                                                  
 lstm_91 (LSTM)                 (None, 50, 100)      80400       ['lstm_90[0][0]']                
                                                                                           

In [85]:
input_tensor = Input(shape=(n_steps), dtype='float32')
x_0 = input_tensor
x = tf.expand_dims(x_0, axis=-1)
# Dependiendo de cual sea el output deberemos cambiar el valor 'return_sequences'
for _ in range(5):
    x = LSTM(100, activation='tanh', recurrent_activation='sigmoid', return_sequences=True)(x)
x = LSTM(200, activation='tanh', recurrent_activation='sigmoid', return_sequences=False)(x)

x = layers.Flatten()(x)
x = layers.Dense(256, activation=tf.nn.leaky_relu)(x)
x = layers.Dense(100, activation=tf.nn.leaky_relu)(x)
out_reg = layers.Dense(1, activation=None)(x)
out_class= layers.Dense(1, activation='softmax')(x)
model = Model(input_tensor, outputs=[out_class,out_reg])
print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=['mse','binary_crossentropy'],metrics=['mse', 'mean_absolute_error',r2_keras])

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 tf.expand_dims_17 (TFOpLambda)  (None, 50, 1)       0           ['input_18[0][0]']               
                                                                                                  
 lstm_102 (LSTM)                (None, 50, 100)      40800       ['tf.expand_dims_17[0][0]']      
                                                                                                  
 lstm_103 (LSTM)                (None, 50, 100)      80400       ['lstm_102[0][0]']               
                                                                                           

In [ ]:
history=model.fit(X_train_sequence, [y_train_sequence,ytrain2], epochs=20, batch_size=30,steps_per_epoch=200, shuffle=False, verbose=1, validation_data=(X_valid_sequence,y_valid),validation_steps=10)

Epoch 1/20
200/200 [==============================] - 88s 440ms/step - loss: 8.6767 - dense_71_loss: 1.0699 - dense_70_loss: 7.6068 - dense_71_mse: 1.0699 - dense_71_mean_absolute_error: 0.9615 - dense_71_r2_keras: -6.8561 - dense_70_mse: 58.9048 - dense_70_mean_absolute_error: 7.6587 - dense_70_r2_keras: -243.6167 - val_loss: 11.8987 - val_dense_71_loss: 0.7909 - val_dense_70_loss: 11.1078 - val_dense_71_mse: 0.7909 - val_dense_71_mean_absolute_error: 0.7284 - val_dense_71_r2_keras: -2.0636 - val_dense_70_mse: 62.4704 - val_dense_70_mean_absolute_error: 7.8873 - val_dense_70_r2_keras: -242.0068
Epoch 2/20
200/200 [==============================] - 76s 382ms/step - loss: 8.6720 - dense_71_loss: 1.0703 - dense_70_loss: 7.6017 - dense_71_mse: 1.0703 - dense_71_mean_absolute_error: 0.9617 - dense_71_r2_keras: -6.8071 - dense_70_mse: 58.8993 - dense_70_mean_absolute_error: 7.6584 - dense_70_r2_keras: -244.2560 - val_loss: 11.8987 - val_dense_71_loss: 0.7909 - val_dense_70_loss: 11.1078 - v

In [81]:
history = model.fit(X_train_sequence, y_train, epochs=20, batch_size=30, steps_per_epoch=200, shuffle=False, verbose=1, validation_data=(X_valid_sequence,y_valid),validation_steps=10)

Epoch 1/20


KeyboardInterrupt: ignored

In [ ]:
show_history(history, metrics=['mse', 'mean_absolute_error','r2_keras'])